<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

_by Grayson Badgley, Oct 2021_

This notebook annotates data entry underlying the calculation of wood product
storage factors for four projects that have been burned in recent years.

The full carbon consequences of unintentional reversals require taking into
account salvaged timber that ends up in wood products. This requires calculating
several conversion factors. First, of all reversed carbon (below/above), what
fraction is merchantable? Then from that merchantable carbon, what fraction of
it ends up in wood products (both in-use and landfilled) that persist for 100
years?

Thankfully, the four projects we're interested in provide the data needed to
directly estimate each of these transitions. Each OPDR makes clear distinctions
between "havested carbon" (which explicitly includes belowground!) and
"merchentable carbon" (boles only, delivered to mill). This means we can
seamlessly translate from loses in IFM-1 (which includes above and below) to
wood products.


In [ ]:
#%load_ext nb_black

from dataclasses import dataclass
import json

import fsspec
import pandas as pd

CACHE_DATA = False


@dataclass
class BaselineWoodProducts:
    dtm: int
    in_use: int
    landfill: int
    frac_merch: float

    def get_landfill_storage_factor(self):
        return self.landfill / self.dtm

    def get_inuse_storage_factor(self):
        return self.in_use / self.dtm

    def get_summary(self):
        return {
            "lf_frac": round(self.get_landfill_storage_factor(), 4),
            "inuse_frac": round(self.get_inuse_storage_factor(), 4),
            "frac_merch": round(self.frac_merch, 4),
        }


records = {}

## ACR255

Attachment G/H of the initial OPDR lists trees harvested at 937,332.63 and DTM
as 440,561. This works out to 47 percent, which is lower than the other three
projects examined.


In [ ]:
bwp = BaselineWoodProducts(
    dtm=440_561.25,
    in_use=114_683,
    landfill=82_226,
    frac_merch=440_561.25 / 937_332.63,
)
records["acr255"] = bwp.get_summary()

## ACR260

Page 57 of the supplemental OPDR has some hints about the fraction of harvested
timber that ends up merchantable. It just reports the annualized total harvest
as 63,693 tCO2e per year. Page 58 reports DTM at 34,405. Baseline in use and
landfill do not vary in the currently submitted OPDRs.


In [ ]:
bwp = BaselineWoodProducts(
    dtm=34_405, in_use=9_239, landfill=6_372, frac_merch=34_405.5 / 63_693.5
)

records["acr260"] = bwp.get_summary()

## ACR273

Page 53 of the initial OPDR gives us what we need! Harvest live is 561,351 and
harvest merch is 300,972. But we can do even better than that. Table 11 gives us
per decade breakdowns. To be conservative, we'll take the maximum ratio between
Removals/AGBG, which works out to be 56.96 percent -- this is just slightly (~4
percent) raito of means.


In [ ]:
bwp = BaselineWoodProducts(
    dtm=300_972, in_use=80_113, landfill=56_777, frac_merch=0.57
)
records["acr273"] = bwp.get_summary()

## CAR1102

Page 43 of the initial OPDR reports Harvested Live and Harvest merchentable as
8657 and 5425. Rather than use the mean values, we'll actually take the maximum
ratio of Cut-All and Cut-Merch from Table 14, which works out to be 64.5 percent


In [ ]:
bwp = BaselineWoodProducts(
    dtm=5_425, in_use=1_513, landfill=964, frac_merch=0.645
)
records["car1102"] = bwp.get_summary()

In [ ]:
if CACHE_DATA:
    fn = "wood_product_storage_factors.json"

    out_dir = "gcs://carbonplan-buffer-analysis"

    out_fn = "/".join([out_dir, fn])
    with fsspec.open(out_fn, "w") as f:
        json.dump(records, f, indent=2)